<a href="https://colab.research.google.com/github/amengemeda/IS-Project-2/blob/main/mT5/Amharic_Hate_Speech_Classification_(mT5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.4 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.0 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 140 kB 66.7 MB/s 
     |████████████████████████████████| 1.1 MB 68.4 MB/s 
     |████████████████████████████████| 212 kB 70.9 MB/s 
     |████████████████████████████████| 127 kB 61.6 MB/s 
     |████████████████████████████████| 271 kB 76.0 MB/s 
     |████████████████████████████████| 144 kB 70.6 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from datasets import Dataset
import datasets

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from transformers import pipeline
# from transformers import AutoModel, BertTokenizerFast
from transformers import AutoTokenizer

In [ ]:
#mount google drive to access the dataset directly from the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#fetch the dataset from the drive
Labels=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Labels.txt',header=None)
Posts=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Posts.txt',header=None)

In [ ]:
Labels.columns = ["label"]
Posts.columns = ["post"]

In [ ]:
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1]) 

In [ ]:
Labels.head(10)

,label
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [ ]:
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [ ]:
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)


In [ ]:
Merged.head(5)

,label,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...


In [ ]:
Merged

,label,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...,...
29995,1,በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996,0,ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997,0,እስከ አሁን አንተ ብቻ ነው በ መፅሀፍ ያልቻልከው አንተም ታሪክ እ...
29998,1,ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...


In [ ]:
#Divide the dataset into train and test categories 
msk = np.random.rand(len(Merged)) < 0.8
train_dataset = Merged[msk]
test_dataset = Merged[~msk]

In [ ]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [ ]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [ ]:
(test_dataset)

Dataset({
    features: ['label', 'post', '__index_level_0__'],
    num_rows: 5979
})

In [ ]:
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")

In [ ]:
#combine to the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [ ]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'post'],
        num_rows: 24017
    })
    test: Dataset({
        features: ['label', 'post'],
        num_rows: 5983
    })
})

In [ ]:
# len(test_dataset)

In [ ]:
from transformers import AutoTokenizer,T5Tokenizer, T5ForConditionalGeneration, MT5ForConditionalGeneration, AutoModelForSequenceClassification


In [ ]:
# model_name = "google/mt5-base" # you can specify the model 

In [ ]:
model_name = "google/mt5-base" # you can specify the model 

In [ ]:
#loading a tokenizer from the pretrained model
# tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [ ]:
tokenizer('ስምህ ማነው')

{'input_ids': [259, 1706, 37755, 5218, 51663, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import T5Model

In [ ]:
model = T5Model("mt5","google/mt5-base")

TypeError: ignored

In [ ]:
#Loading the model
model = MT5ForConditionalGeneration.from_pretrained(model_name,num_labels=2)

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

In [ ]:
# from transformers import MT5Model, T5Tokenizer

In [ ]:
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("google/mt5-base", num_labels=2),

In [ ]:
#Have a tokenizer function that uses the tokenizer of the pretrained model
def tokenize_function(examples):
    return tokenizer(examples["post"], padding="max_length", truncation=True)

In [ ]:
#Tokenize all the data
tokenized_datasets = main_dataset.map(tokenize_function)


Parameter 'function'=<function tokenize_function at 0x7f10b21a3b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/24021 [00:00<?, ?ex/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/5979 [00:00<?, ?ex/s]

In [ ]:
#For Experimental reason, few data(size of 50) is used
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50))

In [ ]:
# pd.DataFrame(small_eval_dataset)
small_eval_dataset

Dataset({
    features: ['label', 'post', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [ ]:
small_train_dataset

Dataset({
    features: ['label', 'post', 'input_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
#load training hyperparameters(Arguments)
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
from transformers import T5Args

ImportError: ignored

In [ ]:
training_args = T5Args()

NameError: ignored

In [ ]:
import numpy as np
from datasets import load_metric

In [ ]:
#
metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer


In [ ]:
print(small_train_dataset[-1])

{'label': 1, 'post': 'አቦ ይመችህ የዘገምተኞቹ አስተሳሰብ ካንተ በ አመት ይዘግማል', 'input_ids': [1876, 9131, 92267, 3264, 1973, 1237, 8183, 3232, 1116, 5359, 103527, 36434, 40966, 3562, 22214, 259, 2401, 90748, 1365, 66782, 986, 3303, 8183, 1840, 169208, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: post. If post are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 200
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 75


ValueError: ignored

In [ ]:
trainer.evaluate()